In [38]:
import pandas as pd
import numpy as np
import random

In [46]:
def get_energy():
    energy_2018 = pd.read_csv("data/nordpool2018_clean.csv")
    energy_2017 = pd.read_csv("data/nordpool2017_clean.csv")
    energy_2016 = pd.read_csv("data/nordpool2016_clean.csv")
    energy = pd.concat([energy_2017, energy_2016, energy_2018])
    weather = pd.read_csv("data/weather1.csv")
    energy['date'] = pd.to_datetime(energy['date'])
    energy = energy.sort_values(by=['date'])
    return energy


def get_train_test(data):
    return data[:0.8*len(data)], data[0.8*len(data):]
def get_training_samples(traindata):
    

In [47]:
energy = get_energy()

twosix = energy[energy['date'] == '2017-01-01']
twosixh = twosix[twosix['hour'] == 10]
def predict_at(l1, l2, energy, year, month, day, hour):
    yesterday = energy[energy['date'] == str(year) + '-' + str(month) + '-' + str((day - 1))]
    twentyfour = yesterday[yesterday['hour'] == hour]
    week = energy[energy['date'] == str(year) + '-' + str(month) + '-' + str(day - 7)]
    weekhour = week[week['hour'] == hour]
    return (twentyfour['price'].values * l1 + weekhour['price'].values * l2)[0]
    #return (twentyfour['price'].values * 0.8 + weekhour['price'].values * 0.2)[0]

def predict_day(l1, l2, energy, year, month, day):
    prices = []
    for hour in range(24):
        prices.append(predict_at(l1, l2, energy, year, month, day + 1, hour))
    return prices

def get_day_prices(energy, year, month, day):
    prices = energy[energy['date'] == str(year) + '-' + str(month) + '-' + str(day)]
    return prices['price'].values

In [73]:
rand = random.Random()
mses = []
for i in range(1000):
    month = (int)(rand.uniform(1,10))
    day = (int)(rand.uniform(10,28))
    year = (int)(rand.uniform(2016,2018))
#    print(year, month, day)
    predicted = predict_day(1.0, 0.0, energy, year, month, day)
    real = get_day_prices(energy, year, month, day)
    mse = ((np.array(predicted) - np.array(real))**2).mean()
    mses.append(mse)
#    print('real', real)
#    print('predicted: ', predicted)
    
#    print("MSE: ", mse)
mses.sort()
print(mses[(int)(len(mses) * 0.5)])
#print('Mean mean square error', np.mean(mses))

2483.8668500000003
